In [12]:
#all the transformers for the Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

#Tackling Ticket to make TktNum and TktPre and dropping ticket column
class Tackling_Ticket(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
        #appending an 'X' for tickets with no prefix and making two new columns
        x2['Ticket'] = x2['Ticket'].apply(lambda x: x.replace(' ','', 1) if len(x.split()) == 3 else x)
        x2['Ticket'] = x2['Ticket'].apply(lambda x: ('X ' + x ) if len(x.split()) == 1 else x)
        x2[['TktPre', 'TktNum']] = x2.Ticket.str.split(' ', expand = True)
        x2["TktNum"] = x2["TktNum"].replace('LINE', 0)
        x2.drop(['Ticket'], axis = 1, inplace = True)
        return x2

#Divind TktPre in categories so we can use dummies on them
class categorizing_TktPre(BaseEstimator, TransformerMixin): 
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
       # print("inside", x2)
        a = x2.TktPre.value_counts().index
        cat1, cat2, cat3 = a[0], a[1:5], a[5:]
        x2["TktPre"] = x2["TktPre"].apply(lambda x: (x in cat1 and 'cat1') or (x in cat2 and 'cat2') or 'cat3')
        return x2

#making catgories for fare 
class categorizing_Continuous(BaseEstimator, TransformerMixin): 
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
        #labels = np.arange(1,21)
        x2 = pd.DataFrame(x2, columns = num_attribs)
        x2['Fare']= x2['Age'].fillna(x2['Age'].mean())
        x2["Fare"] = pd.qcut(x2.Fare, q = 5, labels = False, duplicates = 'drop')
        return x2

def myfun(row):
    if (math.isnan(row['Age']) and row['Survived'] == 1):
        return 0.75
    elif (math.isnan(row['Age']) and row['Survived'] == 0):
        return 70
    else:
        return row['Age']  
    
#Filling in the age valeus for nan. age 70 has 0 survival rate and 0.75 has 1
class Age_impute(BaseEstimator, TransformerMixin): 
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
        if (x2.shape[0] == 891 ):
            x2['Survived'] = tit_data2["Survived"]
            #print("YES")
            x2['Age'] = x2.apply(lambda row: myfun(row), axis = 1)
            #print(x2.shape)
            x2.drop('Survived', axis = 1, inplace = True)
            return x2
        else:
            x2 = x2.fillna(x2.median())
            return x2

class Age_impute_closestRows(BaseEstimator, TransformerMixin): 
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, dataset, y=None):
            index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)
            for i in index_NaN_age :
                age_med = dataset["Age"].median()
                age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
                if not np.isnan(age_pred) :
                    dataset['Age'].iloc[i] = age_pred
                else :
                    dataset['Age'].iloc[i] = age_med
            return dataset

        
#Taking first alphabet of the cabin and 'M' for empty columns
class Prep_Cabin(BaseEstimator, TransformerMixin): 
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
        key = lambda x: 'M' if pd.isnull(x['Cabin']) else x['Cabin'][0]
        x2['Cabin'] = x2.apply(key, axis = 1)
        return x2
       

In [13]:
import numpy as np
import scipy
import pandas as pd
import math
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from pandas.api.types import CategoricalDtype



TITANIC_TRAINING_PATH = "datasets/Titanic/train.csv"
TITANIC_TESTING_PATH = "datasets/Titanic/test.csv"


import warnings
warnings.filterwarnings('ignore')

def load_titanic_data(titanic_path = TITANIC_TRAINING_PATH):
    return pd.read_csv(titanic_path)

tit_test_data = load_titanic_data(TITANIC_TESTING_PATH)
tit_data = load_titanic_data()
tit_data2 = tit_data.copy()

y_train = (tit_data2["Survived"]).values

tit_data2_arr = tit_data2.values
#randomizing
shuffle_index = np.random.permutation(891)    #0 to 890
tit_data2_arr, y_train = tit_data2_arr[shuffle_index], y_train[shuffle_index]
tit_data2 = pd.DataFrame(tit_data2_arr, columns = tit_data2.columns)



    
#DIviding features in numerical and categorical for Pipelines
num_attribs = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
cat_attribs = ['Sex', 'Embarked', 'Ticket', 'Cabin']

dummies_columns = ['Sex_F','Sex_M', 'Em_C', 'Em_Q', 'Em_S',  'Cab_A', 'Cab_B','Cab_C','Cab_D','Cab_E' ,'Cab_F' ,'Cab_G' ,'Cab_M' ,'Cab_T' ,'Tkt_cat1', 'Tkt_cat2', 'Tkt_cat3' ,'TktNum']
columns_prepared = num_attribs + dummies_columns



#making dummies of the features
#using individual columns for dummies because thats the only way to ensure dummies remain same over test and train
class making_dummies(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x2, y=None):
        return self
    def transform(self, x2, y=None):
        df_dum = list()
        string_feats = x2[['Sex', 'Embarked', 'Cabin', 'TktPre']]
        
        cats = {string_feats.columns[0]: ['female', 'male'], string_feats.columns[1]: ['C', 'Q', 'S'], string_feats.columns[2]: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'M', 'T'], string_feats.columns[3]: ['cat1', 'cat2', 'cat3'] }
        
        for column in string_feats:
            string_feats[column] = string_feats[column].astype('category', categories = cats[column])
            data = pd.get_dummies(string_feats[column])
            data_parse = data.values.shape[1]
            for i in range(data_parse):
                lst = [item[i] for item in data.values]
                df_dum.append(lst) 
        dumns = [item for items in cats.values() for item in items]
        df_dum = pd.DataFrame( np.array(df_dum).T, columns = dumns)
        df_dum = pd.concat([df_dum, x2['TktNum']], axis = 1)
        
        return df_dum


num_pipeline = Pipeline([
        ('imputer', Age_impute()),
        #('imputer', SimpleImputer(strategy = 'mean')),
        ('categorizing_continuous', categorizing_Continuous()),
        ('std_scaler', StandardScaler())
    ])


cat_pipeline = Pipeline([
        ('Tackling_Ticket', Tackling_Ticket()),
        ('categorizing TktPre', categorizing_TktPre()),
        ('Cabin_prep', Prep_Cabin()),
        ('Getting_dummies', making_dummies() )
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

data_prepared = full_pipeline.fit_transform(tit_data2)
X_train = pd.DataFrame(data_prepared, columns = columns_prepared)
print(X_train.shape)

testing_data_prepared = full_pipeline.fit_transform(tit_test_data)
X_test = pd.DataFrame(testing_data_prepared, columns = columns_prepared)
print(X_test.shape)

(891, 23)
(418, 23)


In [ ]:
#Removing outliers

from collections import Counter
print("training set before removing outliers ", X_train.shape)

#removing otliers from the data
def remove_outlier_rows(f_name_array, given_number, data_name):
    outlier_cols = []
    
    for one_col in f_name_array:
        P1 = np.percentile(data_name[one_col], 25)
        P2 = np.percentile(data_name[one_col], 75)
        IQR = P2 - P1
        outlier_step = 1.5 * IQR
       
        indices_outlier_rows = data_name[(data_name[one_col] > P2 + outlier_step) | (data_name[one_col] < P1 - outlier_step)].index
        #print(X_train[(X_train[one_col] > P2 + IQR) | (X_train[one_col] < P1 - IQR)])
        outlier_cols.extend(indices_outlier_rows)
       
    check = Counter(outlier_cols)
    
    rows_to_remove = [k for k,v in check.items() if v > given_number]
    
    print([rows_to_remove])
    data_name.drop(rows_to_remove, inplace = True)
    return rows_to_remove

rows_removed_train = remove_outlier_rows(["Age", "Parch", "SibSp", "Fare"], 2, X_train)

y_train = np.delete(y_train, rows_removed_train, 0)

print("training set after removing outliers ",X_train.shape)
print("training labels shape ", y_train.shape) 

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict

forest_clf = RandomForestClassifier(random_state = 42)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
params = {
    'n_estimators': randint(50, 100),
     'max_features': randint(1, 24),
     'oob_score' : [True],
     'criterion': ['gini', 'entropy']
}

newgridsearch = RandomizedSearchCV(forest_clf, param_distributions = params, n_iter=20, cv = 5, scoring='accuracy')
r_grid_search = newgridsearch.fit(X_train, y_train)

final_model = newgridsearch.best_estimator_
frst_scrs = cross_val_score(final_model, X_train, y_train, cv = 3, scoring = "accuracy")

frst_scrs.mean()*100



83.61391694725027

In [ ]:
#running on the test_Data
final_model = newgridsearch.best_estimator_
letsee = final_model.predict(X_test)
letsee

In [ ]:
fdata = np.vstack((tit_test_data.PassengerId, letsee)).ravel('F')
fdata = fdata.reshape(-1,2)
results_final = pd.DataFrame(fdata)
results_final
file_name = 'testing.csv'
results_final.to_csv(file_name, header = ['PassengerId', 'Survived'], index = False)


In [ ]:
def making_bar(attribute):
    X_train['Survived'] = pd.Series(y_train)
    survivability = X_train.groupby([attribute])["Survived"].mean()
    values = X_train.groupby([attribute]).groups.keys()
    plt.bar(values, survivability)
    plt.ylabel('Survivability')
    plt.xlabel(attribute)
making_bar('Cabin')

#plotting Cabin
key = lambda x: 'M' if pd.isnull(x['Cabin']) else x['Cabin'][0]
a = tit_test_data.apply(key, axis = 1)

a1 = a.value_counts().index
a2 = a.value_counts().values

plt.bar(a1, a2)